### Getting started with leanclient

leanclient allows for easy interaction with the [lean4 theorem prover](https://github.com/leanprover/lean4) using the [Language Server Protocol](https://microsoft.github.io/language-server-protocol/specifications/lsp/3.17/specification/).

This notebook shows a minimal example for how to setup and use leanclient.

- leanclient does **not create/manage lean projects**. You should create your own project, e.g. in vscode, and then point leanclient at the root folder (where lakefile.toml is located). See below for an example setup.

- Running this on Google Colab is **slow for larger projects** (e.g. including mathlib). In general, performance is reduced in notebooks.

Check out the [documentation](https://leanclient.readthedocs.io/en/latest/) and the [github repository](https://github.com/oOo0oOo/leanclient) for more information.

In [1]:
PROJECT_NAME = "LeanProject"
PROJECT_PATH = PROJECT_NAME + "/"
LEAN_VERSION = "stable"

In [ ]:
# Setup elan, lean, and a new project
!curl https://raw.githubusercontent.com/leanprover/elan/master/elan-init.sh -sSf | sh -s -- -y

import os

os.environ["PATH"] = os.path.expanduser("~/.elan/bin") + ":" + os.environ["PATH"]

!elan toolchain install leanprover/lean4:{LEAN_VERSION}
!elan default leanprover/lean4:{LEAN_VERSION}

!lake new {PROJECT_NAME}
%cd {PROJECT_NAME}
!lake build
%cd ..

In [ ]:
# Install leanclient
!pip install leanclient

In [4]:
from pprint import pprint
import leanclient as lc

# Create client and point it to the root folder of the project.
# This is where lakefile.toml is located.
client = lc.LeanLSPClient(PROJECT_PATH)

# Use a SingleFileClient for simple interactions with a file.
# This file is a very simple template created during project init.
sfc = client.create_file_client("LeanProject/Basic.lean")

# Query the language server!
symbols = sfc.get_document_symbols()
tokens = sfc.get_semantic_tokens()
term_goal = sfc.get_term_goal(0, 20)

print("Symbols:")
pprint(symbols)

print("\nSemantic tokens:")
pprint(tokens)

print("\nTerm goal:")
pprint(term_goal)

Symbols:
[{'kind': 'method',
  'name': 'hello',
  'range': {'end': {'character': 20, 'line': 0},
            'start': {'character': 0, 'line': 0}},
  'selectionRange': {'end': {'character': 9, 'line': 0},
                     'start': {'character': 4, 'line': 0}}}]

Semantic tokens:
[[0, 0, 3, 'keyword']]

Term goal:
{'goal': '⊢ String',
 'range': {'end': {'character': 20, 'line': 0},
           'start': {'character': 13, 'line': 0}}}


In [5]:
# Start with an incomplete theorem, update the file contents (only in LSP not on disk)
theorem = """theorem my_first_proof (n : Nat) : n = n := by
  sorry"""
sfc.update_file([lc.DocumentContentChange(text=theorem, start=(0, 0), end=(2, 0))])

# Check what we need to prove (cursor after 'by' on line 1, column 2)
goal = sfc.get_goal(1, 2)
print("Current goal:")
print(goal["goals"][0])

# Complete the proof with 'rfl'
theorem_complete = """theorem my_first_proof (n : Nat) : n = n := by
  rfl"""
sfc.update_file(
    [lc.DocumentContentChange(text=theorem_complete, start=(0, 0), end=(1, 7))]
)

# Check for errors
diagnostics = sfc.get_diagnostics()
print("\nNo diagnostics after completing proof:")
pprint(diagnostics)

# Try a more interesting example - incomplete proof
theorem2 = """theorem add_comm_partial (a b : Nat) : a + b = b + a := by
  sorry"""
sfc.update_file([lc.DocumentContentChange(text=theorem2, start=(0, 0), end=(1, 5))])

goal2 = sfc.get_goal(1, 2)
print("\nMore interesting goal (commutativity of addition):")
print(goal2["goals"][0])

# Diagnostics now include a warning
diagnostics2 = sfc.get_diagnostics()
print("\nDiagnostics with sorry:")
pprint(diagnostics2)

Current goal:
n : Nat
⊢ n = n

No diagnostics after completing proof:
[]

More interesting goal (commutativity of addition):
a b : Nat
⊢ a + b = b + a

Diagnostics with sorry:
[{'fullRange': {'end': {'character': 24, 'line': 0},
                'start': {'character': 8, 'line': 0}},
  'message': "declaration uses 'sorry'",
  'range': {'end': {'character': 24, 'line': 0},
            'start': {'character': 8, 'line': 0}},
  'severity': 2,
  'source': 'Lean 4'}]


In [6]:
# Always close the client when you're done.
client.close()

### Next Steps

Check out the [documentation](https://leanclient.readthedocs.io/en/latest/) for more information on leanclient!